###**Gaussian Naive Bayes is used to give predictions with input data as conitnuous values, K-fold cross validation is used here for train-test split**

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df=pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data",header=None)

In [ ]:
names=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country','salary']
df.columns=names

In [ ]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


Missing values filled

In [ ]:
miss_col=[]
for column in df.columns:
    try:
        data= df[df[column]==' ?']
        if data.shape[0]>0:
            miss_col.append(column)
            
    except:
        continue;

for i in miss_col:
    s=pd.Series(df[i])
    s.replace(' ?',s.mode()[0], inplace=True)

Continuous attributes of Adult dataset is taken for Gaussain Naive Bayes

In [ ]:
df_numeric=df[df._get_numeric_data().columns]

In [ ]:
df_numeric['salary']=df['salary']
df_numeric.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,salary
0,39,77516,13,2174,0,40,<=50K
1,50,83311,13,0,0,13,<=50K
2,38,215646,9,0,0,40,<=50K
3,53,234721,7,0,0,40,<=50K
4,28,338409,13,0,0,40,<=50K


In [ ]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=5)
kf

KFold(n_splits=5, random_state=None, shuffle=False)

K-fold algorithm is used that splits data in K partitions. Out of these k (here 5) partitions 1 is the test and rest train. Running the algorithm to predict test data, 'k' times, every partition atleast once becomes test data, ensuring no bias in the results

K-fold cross validation gives list of test_indexes and train_indexes, splits dataset as many times as value of n_splits so each row is eventially in some test data. For each set of training and testing data we get a predict_list of predictions of test data. We have an empty series to which we append these predictions at the indexes crorresponding to test_indexes for that test data. We could sort it later. Now this result list is compared with Output Column of our entire data to get accuracy.

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

results=pd.Series()
                     
for train_index, test_index in kf.split(df_numeric):
    XTrain, xtest, YTrain, ytest = df_numeric.drop(columns='salary').iloc[train_index], df_numeric.drop(columns='salary').iloc[test_index], df_numeric['salary'].iloc[train_index], df_numeric['salary'].iloc[test_index]
    gnb.fit(XTrain,YTrain)
    
    predict_list=gnb.predict(xtest)
    print(predict_list)
    
    indexes=list(test_index)    
    res=pd.Series(predict_list,index=indexes)
    results=results.append(res)
    
    from sklearn import metrics
    print("Accuracy:",(metrics.accuracy_score(ytest, predict_list)*100),"%")
    
print(results)    
from sklearn import metrics
print("Accuracy:",(metrics.accuracy_score(df['salary'], results)*100),"%")

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  after removing the cwd from sys.path.


[' <=50K' ' <=50K' ' <=50K' ... ' <=50K' ' <=50K' ' <=50K']
Accuracy: 79.36434822662368 %
[' <=50K' ' <=50K' ' <=50K' ... ' <=50K' ' <=50K' ' <=50K']
Accuracy: 79.43796068796068 %
[' <=50K' ' <=50K' ' <=50K' ... ' <=50K' ' >50K' ' <=50K']
Accuracy: 79.99078624078624 %
[' <=50K' ' <=50K' ' <=50K' ... ' <=50K' ' <=50K' ' <=50K']
Accuracy: 79.17690417690417 %
[' <=50K' ' <=50K' ' <=50K' ... ' <=50K' ' <=50K' ' >50K']
Accuracy: 79.54545454545455 %
0         <=50K
1         <=50K
2         <=50K
3         <=50K
4         <=50K
          ...  
32556     <=50K
32557     <=50K
32558     <=50K
32559     <=50K
32560      >50K
Length: 32561, dtype: object
Accuracy: 79.50308651454193 %


This is an example of how results from each partition are combined according to indexes

In [ ]:
f=pd.Series()
g=pd.Series([10,11,12,13],index=[1,2,3,4])
print(g)
f=f.append(g)
print(f)
h=pd.Series([10,11,12,13],index=[21,32,13,8])
f=f.append(h)

1    10
2    11
3    12
4    13
dtype: int64
1    10
2    11
3    12
4    13
dtype: int64


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  """Entry point for launching an IPython kernel.
